In [0]:
%sql
-- Verficações da camada BRONZE pós-pipeline:

SHOW TABLES IN ampev.bronze;

In [0]:
%sql
-- Verificar se existem registros pasta estabelecimentos:

SELECT COUNT(*) FROM ampev.bronze.estabelecimentos;




In [0]:
%sql
-- Verificar se existem registros pasta pedidos:

SELECT COUNT(*) FROM ampev.bronze.pedidos;




In [0]:
%sql
-- Verificar se o metadata foi salvo corretamente da tabela estabelecimentos:

SELECT _source_file, _ingest_ts
FROM ampev.bronze.estabelecimentos
LIMIT 5;




In [0]:
%sql
-- Verificar se o metadata foi salvo corretamente da tabela pedidos:

SELECT _source_file, _ingest_ts
FROM ampev.bronze.pedidos
LIMIT 5;




In [0]:
%sql
-- Verificar histórico Delta :

DESCRIBE HISTORY ampev.bronze.estabelecimentos;




In [0]:
%sql
-- Verificar histórico Delta :

DESCRIBE HISTORY ampev.bronze.pedidos;




In [0]:
# Auditoria Completa:

from pyspark.sql.utils import AnalysisException

CATALOG = "ampev"
SCHEMA  = "bronze"
LANDING = "/Volumes/ampev/bronze/landings"

TABLES = [
    (f"{CATALOG}.{SCHEMA}.estabelecimentos", f"{LANDING}/_checkpoints/estabelecimentos"),
    (f"{CATALOG}.{SCHEMA}.pedidos",          f"{LANDING}/_checkpoints/pedidos"),
]

def table_exists(full_name: str) -> bool:
    try:
        spark.table(full_name)
        return True
    except AnalysisException:
        return False

def count_rows(full_name: str) -> int:
    return spark.sql(f"SELECT COUNT(*) AS c FROM {full_name}").collect()[0]["c"]

def has_metadata(full_name: str) -> bool:
    # verifica se _source_file existe e tem pelo menos 1 valor não-nulo
    try:
        r = spark.sql(
            f"SELECT SUM(CASE WHEN _source_file IS NOT NULL AND _source_file <> '' THEN 1 ELSE 0 END) AS ok "
            f"FROM {full_name}"
        ).collect()[0]["ok"]
        return r is not None and r > 0
    except Exception:
        return False

def last_history(full_name: str) -> dict:
    # pega 1 linha do DESCRIBE HISTORY
    rows = spark.sql(f"DESCRIBE HISTORY {full_name}").limit(1).collect()
    if not rows:
        return {}
    row = rows[0].asDict()
    # campos variam um pouco por runtime; vamos pegar os mais úteis quando existirem
    return {
        "timestamp": row.get("timestamp"),
        "operation": row.get("operation"),
        "operationMetrics": row.get("operationMetrics"),
        "userName": row.get("userName"),
    }

def checkpoint_exists(path: str) -> bool:
    try:
        return len(dbutils.fs.ls(path)) > 0
    except Exception:
        return False

def audit_table(full_name: str, checkpoint_path: str) -> dict:
    result = {
        "table": full_name,
        "exists": False,
        "row_count": None,
        "metadata_ok": None,
        "history": None,
        "checkpoint_ok": None,
        "errors": [],
    }

    if not table_exists(full_name):
        result["errors"].append("Tabela não existe no Unity Catalog.")
        return result

    result["exists"] = True

    try:
        result["row_count"] = count_rows(full_name)
        if result["row_count"] == 0:
            result["errors"].append("Tabela existe, mas está vazia (0 linhas).")
    except Exception as e:
        result["errors"].append(f"Falha ao contar linhas: {e}")

    try:
        result["metadata_ok"] = has_metadata(full_name)
        if result["row_count"] and result["row_count"] > 0 and not result["metadata_ok"]:
            result["errors"].append("Linhas existem, mas _source_file não está preenchido (metadata).")
    except Exception as e:
        result["errors"].append(f"Falha ao validar metadata (_source_file): {e}")

    try:
        result["history"] = last_history(full_name)
        if result["history"] and result["history"].get("operation") is None:
            result["errors"].append("Não foi possível ler operação no DESCRIBE HISTORY.")
    except Exception as e:
        result["errors"].append(f"Falha ao ler DESCRIBE HISTORY: {e}")

    # checkpoint é opcional: em availableNow ele costuma existir depois que roda com arquivos
    try:
        result["checkpoint_ok"] = checkpoint_exists(checkpoint_path)
        if result["row_count"] and result["row_count"] > 0 and not result["checkpoint_ok"]:
            result["errors"].append("Linhas existem, mas checkpoint não foi encontrado (verifique path/permissão).")
    except Exception as e:
        result["errors"].append(f"Falha ao validar checkpoint: {e}")

    return result

# --- roda auditoria ---
results = [audit_table(t, ckpt) for t, ckpt in TABLES]

# imprime resumo legível
all_ok = True
for r in results:
    print("\n==============================")
    print("Tabela:", r["table"])
    print("Existe:", r["exists"])
    print("Linhas:", r["row_count"])
    print("Metadata (_source_file) OK:", r["metadata_ok"])
    print("Checkpoint OK:", r["checkpoint_ok"])
    print("History:", r["history"])
    if r["errors"]:
        all_ok = False
        print("ERROS:")
        for e in r["errors"]:
            print(" -", e)

print("\n==============================")
print("PIPELINE OK " if all_ok else "PIPELINE FAIL ")
